Graph-based segmentation by Felzenszwalb is a popular image segmentation algorithm that uses graph theory principles to partition an image into meaningful regions. It was proposed by Pedro F. Felzenszwalb and Daniel P. Huttenlocher in their paper titled "Efficient Graph-Based Image Segmentation" in 2004.

Here's an overview of the algorithm:

Image Representation: The algorithm begins by converting the input image into a graph representation. Each pixel in the image becomes a node in the graph, and weighted edges are defined between adjacent pixels.

Edge Weight Calculation: The weights of the edges connecting the nodes are computed based on the difference in pixel intensity or color between the adjacent pixels. The larger the difference, the higher the weight of the edge.

Graph Segmentation: The algorithm employs a graph-based segmentation technique called minimum spanning trees (MST) to segment the image. The MST connects all the nodes with the minimum total weight while avoiding cycles.

Region Merging: The algorithm iteratively merges the regions created by the MST based on a similarity criterion. Regions with similar average pixel intensities or colors are merged together. This process continues until no more merges can be made, and the final segmentation is obtained.

Parameter Selection: The algorithm involves a parameter called the "scale parameter" that determines the level of detail in the segmentation. This parameter controls the trade-off between over-segmentation and under-segmentation. The optimal value of this parameter depends on the specific image and application, and it may need to be tuned for satisfactory results.

The graph-based segmentation algorithm by Felzenszwalb has been widely adopted due to its efficiency and ability to handle images with varying textures and complex boundaries. It provides a hierarchical segmentation, allowing the user to obtain multiple levels of detail by adjusting the scale parameter.

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from skimage.measure import find_contours
from skimage.measure import regionprops
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=50, sigma=0.1)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Set the maximum segment size threshold
max_segment_size = 1000

# Set the intensity threshold for background segments
background_threshold = 100

# Set the solidity threshold for irregular fragments
solidity_threshold = 0.7

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Calculate the size of the segment (number of pixels)
    segment_size = np.sum(segment_mask)

    # Skip segments above the maximum size threshold
    if segment_size > max_segment_size:
        continue

    # Extract the intensity values of the segment
    segment_intensities = grayscale_image[segment_mask]

    # Check if the segment is predominantly background based on intensity threshold
    if np.mean(segment_intensities) < background_threshold:
        continue

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Calculate the solidity of the segment
    props = regionprops(segment_mask.astype(int))[0]
    solidity = props.solidity

    # Check if the segment has irregular fragments based on solidity threshold
    if solidity < solidity_threshold:
        continue

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)